In [ ]:
import requests
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit
from reportlab.lib.colors import Color
import re
import pandas as pd
import numpy as np
import time

In [ ]:
notas_para_numeros = {
            'A': 0, 'A#': 1, 'Bb': 1, 'B': 2, 'C': 3, 'C#': 4, 'Db': 4, 'D': 5,
            'D#': 6, 'Eb': 6, 'E': 7, 'F': 8, 'F#': 9, 'Gb': 9, 'G': 10, 'G#': 11, 'Ab': 11
        }

def extrair_cifra(url, tom = 0):
    """
    Extrai a cifra de uma música a partir de uma URL do Cifra Club.

    :param url: URL da página da cifra.
    :param js: se será necessário javascript
    :return: String com a cifra extraída.
    """
    try:
        # Faz a requisição HTTP para abrir o link
        print(f"Acessando o link: {url}")
        site = requests.get(url, headers=headers)
        site.raise_for_status()  # Verifica se a requisição foi bem-sucedida

        # Parseia o conteúdo HTML
        soup = BeautifulSoup(site.content, 'html.parser')

        # Remove as tags <span> que contêm a classe 'tablatura'
        for tablatura in soup.find_all('span', class_='tablatura'):
            tablatura.decompose()

        # Encontra todas as tags <pre> que contêm a cifra
        pre_tags = soup.find_all('pre')

        # Extrai o texto, removendo linhas em branco extras
        cifras = []
        for pre in pre_tags:
            linhas = pre.get_text().splitlines()
            linhas_filtradas = [linha for linha in linhas if linha.strip()]
            cifras.append("\n".join(linhas_filtradas))



        return "\n".join(cifras)  # Junta todas as cifras em uma única string
    except Exception as e:
        print(f"Erro ao extrair a cifra: {e}")
        return None


# Gera um PDF com as cifras
def criar_pdf(cifras_com_titulos, output_file="cifra.pdf", fonte="Courier", tamanho_fonte=12):
    pdf = canvas.Canvas(output_file, pagesize=A4)
    largura_pagina, altura_pagina = A4
    margem_esquerda = 50
    margem_direita = largura_pagina / 2 + 20
    centro_x = largura_pagina / 2
    margem_superior = altura_pagina - 50
    margem_inferior = 50
    espacamento_linhas = tamanho_fonte + 2
    espacamento_titulos = 30

    primeira_pagina = True

    for titulo, cifra in cifras_com_titulos:
        if not primeira_pagina:
            pdf.showPage()
        primeira_pagina = False

        pdf.setFont("Helvetica-Bold", 16)
        pdf.drawCentredString(centro_x, margem_superior, titulo)

        pdf.setFont(fonte, tamanho_fonte)
        y_coluna_esquerda = margem_superior - espacamento_titulos
        y_coluna_direita = y_coluna_esquerda
        x_coluna_esquerda = margem_esquerda
        x_coluna_direita = margem_direita
        coluna_atual = 1

        for line in cifra.split("\n"):
            if coluna_atual == 1:
                if y_coluna_esquerda < margem_inferior:
                    coluna_atual = 2
                else:
                    pdf.drawString(x_coluna_esquerda, y_coluna_esquerda, line)
                    y_coluna_esquerda -= espacamento_linhas
            if coluna_atual == 2:
                if y_coluna_direita < margem_inferior:
                    break
                pdf.drawString(x_coluna_direita, y_coluna_direita, line)
                y_coluna_direita -= espacamento_linhas

    pdf.save()
    print(f"PDF salvo como {output_file}")

# Função para criar a coluna de links
def criar_coluna_links(df):
    try:
        def gerar_link(row):
            artista = row[1].lower().replace(" ", "-").replace("'", "").replace("ã", "a")\
                .replace("á", "a").replace("â", "a").replace("é", "e").replace("ê", "e")\
                .replace("í", "i").replace("ó", "o").replace("ô", "o").replace("ú", "u")\
                .replace("ç", "c")
            musica = row[0].lower().replace(" ", "-").replace("'", "").replace("ã", "a")\
                .replace("á", "a").replace("â", "a").replace("é", "e").replace("ê", "e")\
                .replace("í", "i").replace("ó", "o").replace("ô", "o").replace("ú", "u")\
                .replace("ç", "c")

            link = f"https://www.cifraclub.com.br/{artista}/{musica}/"

            if row['Simplificada'] != 0:
                link += "simplificada.html"

            if isinstance(row['Tom'], str) and any(c.isalpha() for c in row['Tom']):
                tom = row['Tom'].strip()
                if tom in notas_para_numeros:
                    numero_tom = notas_para_numeros[tom]
                    link += f"#key={numero_tom}"

            return link

        df['Link'] = df.apply(gerar_link, axis=1)
        print("Coluna de links criada com sucesso!")
        return df

    except Exception as e:
        print(f"Erro ao criar coluna de links: {e}")
        return None

In [ ]:
path = 'repertorio.csv'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0"}
repertorio = pd.read_csv(path)

In [ ]:
repertorio = criar_coluna_links(repertorio)

Coluna de links criada com sucesso!


C:\Users\IFPR - Astorga\AppData\Local\Temp\ipykernel_3508\3450411228.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  artista = row[1].lower().replace(" ", "-").replace("'", "").replace("ã", "a")\
C:\Users\IFPR - Astorga\AppData\Local\Temp\ipykernel_3508\3450411228.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  musica = row[0].lower().replace(" ", "-").replace("'", "").replace("ã", "a")\


In [ ]:
cifras_com_titulos = []

# Itera sobre as linhas do DataFrame
for index, row in repertorio.iterrows():
    link = row["Link"]
    titulo = row["Música"]
    print(f"Processando: {titulo} - {link}")


    cifra = transpose_chords(extrair_cifra(link),1)
    # Extrai a cifra do link
    #if repertorio['Tom'].isna()[index]:
    #    cifra = extrair_cifra(link, tom=1)
    #else:
    #    cifra = extrair_cifra(link)

    # Verifica se a cifra está vazia
    if cifra is None or not cifra.strip():  # Primeiro verifica se é None, depois se está vazia
        print(f"Processando: {titulo} - {link}")
        print(f"Aviso: A cifra para '{titulo}' está vazia ou não foi encontrada. Pulando esta música.")
    else:
        # Adiciona a tupla (título, cifra) à lista
        cifras_com_titulos.append((titulo, cifra))

criar_pdf(cifras_com_titulos, "cifras_la.pdf", 'Helvetica')

Processando: Mulher de Fases - https://www.cifraclub.com.br/raimundos/mulher-de-fases/#key=0
Acessando o link: https://www.cifraclub.com.br/raimundos/mulher-de-fases/#key=0
Processando: Anna Júlia - https://www.cifraclub.com.br/los-hermanos/anna-julia/
Acessando o link: https://www.cifraclub.com.br/los-hermanos/anna-julia/
Processando: Exagerado - https://www.cifraclub.com.br/cazuza/exagerado/
Acessando o link: https://www.cifraclub.com.br/cazuza/exagerado/
Processando: Enquanto Me Beija - https://www.cifraclub.com.br/jao/enquanto-me-beija/
Acessando o link: https://www.cifraclub.com.br/jao/enquanto-me-beija/
Processando: Meninos e Meninas - https://www.cifraclub.com.br/jao/meninos-e-meninas/
Acessando o link: https://www.cifraclub.com.br/jao/meninos-e-meninas/
Processando: Terra de Gigantes - https://www.cifraclub.com.br/engenheiros-do-hawaii/terra-de-gigantes/
Acessando o link: https://www.cifraclub.com.br/engenheiros-do-hawaii/terra-de-gigantes/
Processando: Era Um Garoto Que Como E

In [ ]:
# 1. Abrir o arquivo CSV
df = pd.read_csv('repertorio.csv')

df.to_csv('repertorio.csv', index=False)

In [ ]:
df = pd.read_csv('repertorio.csv')
df

,Música,Artista,Gênero,Bloco,Tom,Simplificada
0,Mulher de Fases,Raimundos,Rock,Boy Like,A,0
1,Anna Júlia,Los Hermanos,Rock,Boy Like,NaN,0
2,Exagerado,Cazuza,MPB,Boy Like,NaN,0
3,Enquanto Me Beija,Jão,MPB,Boy Like,NaN,0
4,Meninos e Meninas,Jão,Rock,Boy Like,NaN,0
5,Terra de Gigantes,Engenheiros do Hawaii,Rock,Boy Like,NaN,0
6,Era Um Garoto Que Como Eu Amava os Beatles e o...,Engenheiros do Hawaii,Rock,Boy Like,NaN,0
7,O Tempo Não Para,Cazuza,MPB,Boy Like,NaN,0
8,Pro Dia Nascer Feliz,Cazuza,MPB,Boy Like,NaN,0
9,O Nosso Amor Gente Inventa,Cazuza,Breve,Boy Like,NaN,0


In [ ]:
cifra

'[Intro] D2  C9  D2  C9\nParte 3 de 5\n( Riff 2 )\n      D2    \nMeu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas inteiras\n       D2\nCabe uma penteadeira\n   C9\nCabe nós dois\n( Riff 2 )\n            D2\nCabe até o meu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas inteiras\n       D2\nCabe uma penteadeira\n   C9\nCabe nós dois\n( Riff 2 )\n            D2\nCabe até o meu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas i

In [ ]:
def transpose_chords(text, shift):
    notas_para_numeros = {
        'A': 0, 'A#': 1, 'Bb': 1, 'B': 2, 'C': 3, 'C#': 4, 'Db': 4, 'D': 5,
        'D#': 6, 'Eb': 6, 'E': 7, 'F': 8, 'F#': 9, 'Gb': 9, 'G': 10, 'G#': 11, 'Ab': 11
    }
    numeros_para_notas = {v: k for k, v in notas_para_numeros.items()}

    def transpose_match(match):
        original_chord = match.group(0)
        root_note = match.group(1)

        if root_note in notas_para_numeros:
            new_note_number = (notas_para_numeros[root_note] + shift) % 12
            new_note = numeros_para_notas[new_note_number]
            return new_note + original_chord[len(root_note):]

        return original_chord  # Retorna sem alterações se não for um acorde válido

    # Ajuste na regex para garantir que acordes complexos sejam detectados corretamente
    chord_pattern = re.compile(r'(?<=\n|\s|\[)([A-G](#|b)?)(m|M|maj|min|dim|aug|sus|add|[0-9]*)\b')
    transposed_text = chord_pattern.sub(transpose_match, text)

    return transposed_text


In [ ]:
cifra

'[Intro] D2  C9  D2  C9\nParte 3 de 5\n( Riff 2 )\n      D2    \nMeu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas inteiras\n       D2\nCabe uma penteadeira\n   C9\nCabe nós dois\n( Riff 2 )\n            D2\nCabe até o meu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas inteiras\n       D2\nCabe uma penteadeira\n   C9\nCabe nós dois\n( Riff 2 )\n            D2\nCabe até o meu amor\n         A          Bm7\nEssa é a última oração\n        G\nPra salvar seu coração\nCoração não é tão simples quanto pensa\nNele cabe o que não cabe na dispensa\n( Riff 1 )\n        D2\nCabe o meu amor\n        C9\nCabem três vidas i

In [ ]:
linhas_filtradas

NameError: name 'linhas_filtradas' is not defined